# Fetch Table from MySQL Server and Upload to S3 as csv

#### Configuration

In [ ]:
# Table to export
table_name = '<table-name>'

# MySQL query
query = f"SELECT * FROM {table_name} LIMIT 50"

# Output CSV file
output_file = f'{table_name}.csv'

# MySQL connection parameters
mysql_config = {
    'host': '<ec2-external-ip>',
    'user': '<database-user>',
    'password': '<database-password>',
    'database': '<database-name>'
}

#### Data Type Conversion Function

In [20]:
def convert_data_type(rows):
    i = 0
    for row in rows:
        row_list = list(row)
        
        j = 0
        for item in row:
            if type(item) in [datetime.date, datetime.datetime]:
                row_list[j] = str(item)

            elif item is None:
                row_list[j] = ''
                
            j += 1
    
        row = tuple(row_list)
        rows[i] = row
        i += 1

    return rows

#### Fetch table rows

In [ ]:
import mysql.connector
import csv
import datetime

# Connect to MySQL
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

# Execute query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Get column names
column_names = [i[0] for i in cursor.description]

# Convert data type
rows = convert_data_type(rows)

# Close connection
cursor.close()
conn.close()

#### Save table as csv

In [ ]:
# Write to CSV
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(column_names)  # Write header
    csvwriter.writerows(rows)  # Write data

print(f"Data exported to {output_file}")

#### Upload to S3

In [27]:
!aws s3 cp user.csv s3://aas-glue/csv/fg_rt/user.csv

Completed 10.1 KiB/10.1 KiB (9.7 KiB/s) with 1 file(s) remaining
upload: .\user.csv to s3://aas-glue/csv/fg_rt/user.csv          
